In [ ]:
# 딥 러닝을 이용한 자연어 처리 입문

In [ ]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
from google.colab import files
from google.colab import drive

warnings.filterwarnings('ignore')


In [ ]:
pwd

'/content'

In [ ]:
drive.mount("/content/Drive/", force_remount=True)

Mounted at /content/Drive/


In [ ]:
df = pd.read_csv('/content/Drive/MyDrive/intent_classifier/mod_intent_model_dataset.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# df.drop(df[df.duplicated()])
df.dropna(inplace=True)
df.drop(df[df.duplicated()].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(언약) 위협하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 부정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 긍정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(선언/위임하기)'].index, axis=0, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
df


,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
3,아닌데 그냥 많이 먹은 거 같은데,(단언) 주장하기
4,그냥 나 뭐 사주기 싫어서 그런 거 같은데.,(단언) 주장하기
...,...,...
1129971,안녕하세유,(표현) 인사하기
1129972,매물 위치와 평수를 알 수 있을까요?,(지시) 질문하기
1129973,잘 진행되고 있나요?,(지시) 질문하기
1129974,네~ 안녕하세요,(표현) 인사하기


In [ ]:
lable_num = len(df['intent'].unique())
lable_num

13

In [ ]:
intet_label = list(df['intent'].unique())

label_dict = {}

for idx, intent_lab in enumerate(intet_label) :
    df.loc[df['intent'] == intent_lab, 'intent'] = idx
    label_dict[idx] = intent_lab

print(label_dict)
df

{0: '(단언) 진술하기', 1: '(지시) 충고/제안하기', 2: '(단언) 주장하기', 3: '(지시) 질문하기', 4: '(지시) 부탁하기', 5: '(단언) 반박하기', 6: '(표현) 감사하기', 7: '(표현) 사과하기', 8: '(지시) 명령/요구하기', 9: '턴토크 사인(관습적 반응)', 10: '(언약) 약속하기(제3자와)/(개인적 수준)', 11: '(언약) 거절하기', 12: '(표현) 인사하기'}


,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,0
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,0
3,아닌데 그냥 많이 먹은 거 같은데,2
4,그냥 나 뭐 사주기 싫어서 그런 거 같은데.,2
...,...,...
1129971,안녕하세유,12
1129972,매물 위치와 평수를 알 수 있을까요?,3
1129973,잘 진행되고 있나요?,3
1129974,네~ 안녕하세요,12


In [ ]:
train_data = pd.DataFrame()

for k,v in label_dict.items():
    len_ = df.loc[df['intent'] == k,'intent'].count()
    if len_ > 20000 :
        train_data = pd.concat([train_data,df.loc[df['intent'] == k].sample(n=10)],ignore_index=True)
    else :
        train_data = pd.concat([train_data,df.loc[df['intent'] == k].sample(n=10)],ignore_index=True)
train_data = train_data.sample(frac=1).reset_index(drop=True)
train_data

,sentence,intent
0,요즘 학원비 엄청 비싸던데,2
1,이야… 오랜만에 연락한번 해보자,1
2,아아 본명이 이은지 였구나,9
3,오 어떤 대화였는데?,11
4,옛날 노래들만 들어 키키,0
...,...,...
125,아 그래도 아니라고는 안 하네 고맙다 ^^,6
126,웅 단톡방에 한번 물어보자,10
127,식당 이름 알려줄게 다음에 배달 시켜서 먹어봐 키키,1
128,그래 조심해서 잘 다녀와,12


In [ ]:
test_data = pd.DataFrame()

for k,v in label_dict.items():
    len_ = df.loc[df['intent'] == k,'intent'].count()
    if len_ > 20000 :
        test_data = pd.concat([test_data,df.loc[df['intent'] == k].sample(n=1)],ignore_index=True)
    else :
        test_data = pd.concat([test_data,df.loc[df['intent'] == k].sample(n=1)],ignore_index=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)
test_data

,sentence,intent
0,키키 근데 그런 공원 한 번밖에 못 봤어,0
1,지하철 타기 불편하지 않니?,3
2,오... 헬스!,9
3,그것도당근에팔아 키키,1
4,맞아 그렇긴하겠다 하지만 이제 남편과 평생살잖아!,2
5,얔 키키 아니라곸 키키,5
6,행님 산청이면 미안 담에 먹자 ㅜ,11
7,맞아 나 링크 보내 줘 나도 살래,4
8,대박 축하해!,12
9,성의 없게 대답 말고 찾아봐,8


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

sent1 = train_data['sentence'].iloc[3]
print(max_seq_len)
print('문 장1 :',sent1)

100
문 장1 : 오 어떤 대화였는데?


In [ ]:
#입력된 전체 데이터에 대해서 이 과정을 진행하는 함수를 만듭니다.
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
    # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True, truncation=True)

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 130/130 [00:00<00:00, 5527.27it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 13/13 [00:00<00:00, 4378.19it/s]


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=4, validation_split=0.2)

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=2)
print("test loss, test acc: ", results)

In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])
    
    print('sentence :',new_sentence)
    print(label_dict[score])


In [ ]:
MODEL_NAME = 'fine-tuned-klue-bert-base'
MODEL_SAVE_PATH = os.path.join("/content/Drive/MyDrive/intent_classifier/debug_model", MODEL_NAME) # change this to your preferred location
print(MODEL_SAVE_PATH)
print('='*50)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)